# 1. Introdução

O conjunto de dados utilizado neste trabalho foi disponibilizado pelo SIGAA e é sigiloso, por isso não haverá links para acessá-lo. O objetivo deste trabalho é clusterizar um conjunto de dados que não possui etiqueta, para isso será usado o algoritmo K-médias e os índices de validação de Dunn e Calinski-Harabasz (CH).

O trecho de código abaixo serviu para carregar o *dataset* no sistema computacional e aferir o número de atributos e amostras disponíveis:

In [1]:
# carregando dataset
import pandas as pd

dataset = pd.read_table("./datasetTC3.dat", header=None)
print("Número de atributos: {}\nNúmero de amostras:  {}".format(dataset.shape[1],dataset.shape[0]))

Número de atributos: 6
Número de amostras:  1701


/home/romulo/Dropbox/Mestrado/[00 old]/[DISCIPLINA] Reconhecimento de Padrões/Trabalhos Computacionais/TC3/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


# 2. Clusterização e análise

A metodologia aplicada para a clusterização e análise foi:
1. Normalizar os dados;
2. Para cada valor distinto de *K*:
   1. Aplicou-se o algoritmo K-médias 100 vezes;
   2. Escolheu-se os protótipos da rodada que produziu a menor soma das distâncias quadráticas (SSD);
   3. Calculou-se os índices de Dunn e CH

Para o cálculo do índice de Dunn foi, após verificada, utilizada a implementação de Joaquim Viegas, disponível em: https://github.com/jqmviegas/jqm_cvi/blob/master/jqmcvi/base.py. Para o índice CH uma função própria foi desenvolvida, inspirada na função de Viegas para índice de Dunn, que o código abaixo apresenta:

In [2]:
from numpy import trace

def ch_eval(points, kmeans):
    ks, N = np.unique(np.sort(kmeans.labels_), return_counts=True)
    x_bar = np.mean(points, axis=0).reshape(-1,1)
    
    l_range = ks.tolist()
    # matriz dispersão entregrupos
    Bk = np.zeros((points.shape[1],points.shape[1]))
    # matriz dispersão intragrupo
    Wk = np.zeros((points.shape[1],points.shape[1]))
    for i in range(len(ks)): # para cada protótip
        wi = kmeans.cluster_centers_[i].reshape(-1,1) # protótipo do cluster 'i'
        temp = wi - x_bar
        Bk += N[i]*np.matmul(temp,temp.T)
        
        Vi = points[kmeans.labels_==i].T
        for l in range(Vi.shape[1]): # para cada elemento da partição 'i'
            temp = Vi[:,l] - wi
            Wk += np.matmul(temp,temp.T)
            
    N = points.shape[0]
    K = len(ks)
    ch = trace(Bk)*(N-K) / ((K-1)*trace(Wk))
    return ch

O código abaixo calcula os índices propostos para diversos valores de $K$ e gera um gráfico dos índices em função do número de agrupamentos:

In [3]:
%%time
import base
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

temp = dataset.astype(np.float64).values
scale = MinMaxScaler().fit(temp) # guardar o objeto scale para fazer a 
                                 # transformação inversa posteriormente
X = scale.transform(temp)

# Hiper-parâmetros
rodadas = 100   # número de rodadas com diferentes inicializações
Ks = [i for i in range(2, 11)] # número de protótipos

# métricas de validação
dunn = [0]*len(Ks)
ch   = [0]*len(Ks)
for K in Ks:
    kmeans = KMeans(n_clusters=K, n_init=rodadas, init='random', n_jobs=-1).fit(X)

    dunn[K-Ks[0]] = base.dunn_fast(X, kmeans.labels_)
    ch[K-Ks[0]]   = ch_eval(X, kmeans)

CPU times: user 6.6 s, sys: 4.7 s, total: 11.3 s
Wall time: 4.41 s


In [4]:
import plotly.offline as plt
import plotly.graph_objs as go

plt.init_notebook_mode(connected=True) # habilitando plot dentro do jupyter notebook


dunn_trace = go.Scatter(x=Ks, y=dunn, mode='lines+markers', name="Índice Dunn")
ch_trace   = go.Scatter(x=Ks, y=ch,   mode='lines+markers', name="Índice CH"  )

data = [dunn_trace, ch_trace]
layout = go.Layout(
    title = "Índice de validação em função de K",
    legend=dict(orientation="h", y=-.05),
    xaxis=dict(title="Número de agrupamentos K"),
    yaxis=dict(title="Índices de validação")
)

fig = go.Figure(data=data, layout=layout)
plt.iplot(fig)

print("K_opt segundo índice de Dunn : {}".format(Ks[np.argmax(dunn)]))
print("K_opt segundo índice de CH   : {}".format(Ks[np.argmax(ch)]))

K_opt segundo índice de Dunn : 2
K_opt segundo índice de CH   : 2


O resultado da simulação aponta que não houve discordância em relação ao $K_{opt}$, mas se houvesse seria natural. Duas métricas diferentes, que quantizam e balanceiam de forma diferente a coesão interna e separação externa dos agrupamentos, podem facilmente discordar no valor ótimo de agrupamentos.

Por fim foi realizada uma análise estatística dos agrupamentos, para tal o código abaixo foi utilizado:

In [5]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

cabecalho = ['Pos. protótipo', 'Mínimo', 'Máximo', 'Mediana', 'Desv. Padrão']

kmeans = KMeans(n_clusters=2, n_init=100, max_iter=300,
                init='random', n_jobs=-1).fit(X)

# calculando estatísticas por agrupamento
for k in range(len(kmeans.cluster_centers_)):
    Vi = X[kmeans.labels_==k].copy()
    Vi = scale.inverse_transform(Vi) # escalonando de volta aos valores originais
    pi = scale.inverse_transform(kmeans.cluster_centers_[k].reshape(1,-1)) # posição do protótipo
    pi = np.asarray(pi).reshape(-1)

    print("Agrupamento {}: ({} objetos)".format(k+1,Vi.shape[0]))
    index = ['Atributo {}'.format(i+1) for i in range(Vi.shape[1])]

    data = np.matrix([pi, np.amin(Vi,axis=0), np.amax(Vi,axis=0), 
                      np.median(Vi,axis=0), np.std(Vi,axis=0)]).T

    df_cluster_k = pd.DataFrame(data, columns=cabecalho, index=[index])
    display(df_cluster_k)
    #print_df(df_cluster_k,'agrupamento{}'.format(k+1), "Agrupamento {}: ({} objetos)".format(k+1,Vi.shape[0]))

Agrupamento 1: (811 objetos)


,Pos. protótipo,Mínimo,Máximo,Mediana,Desv. Padrão
Atributo 1,98.62,0.00,100.00,100.00,7.91
Atributo 2,4.33,0.00,72.67,3.17,4.49
Atributo 3,"1,912.55",38.22,"117,287.30",342.19,"6,589.50"
Atributo 4,2.09,0.21,3.46,1.81,1.05
Atributo 5,2.16,0.27,3.46,2.00,1.01
Atributo 6,"251,644.98",10.00,"976,193.00","135,089.00","275,146.62"


Agrupamento 2: (890 objetos)


,Pos. protótipo,Mínimo,Máximo,Mediana,Desv. Padrão
Atributo 1,99.62,50.00,100.00,100.00,3.02
Atributo 2,4.09,0.00,37.00,3.75,2.68
Atributo 3,"1,009.29",45.34,"24,965.86",365.95,"1,947.01"
Atributo 4,2.24,0.38,3.46,2.34,1.03
Atributo 5,2.28,0.43,3.46,2.34,1.01
Atributo 6,"9,354,056.55","8,000,011.00","9,995,020.00","9,371,503.50","322,709.28"


In [6]:
def print_df(df, nome, descricao):
    print(
        "\\begin{table}[h!]\n"
        "    \captionsetup{width=16cm}%ATENÇÃO: Ajuste a largura do título\n"
        "    \Caption{\label{tab:"+nome+"} "+descricao+"}\n"
        "    \\begin{adjustbox}{width=1\\textwidth}\n"
        "    \small\n"
        +df.to_latex()+
        "    \end{adjustbox}\n"
        "    \Fonte{O autor.}\n"
        "\end{table}")